In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import pytorch_nndct
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_nndct import Inspector
batch_size = 1
# load data
data = pd.read_pickle("RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [1] * 1000  # QPSK = 0
bpsk_labels = [0] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to PyTorch tensor with Long data type
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = torch.from_numpy(labels_combined).long()

# convert 2 PyTorch tensor
data_combined = torch.from_numpy(data_combined).float()

# convert labels 2 NumPy array and then 2 PyTorch tensor
labels_combined = np.array(labels_combined)
labels_combined = torch.from_numpy(labels_combined)

# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # access a single data sample and label
        sample = self.data[idx]
        label = self.labels[idx]
    
        # Convert sample, min_vals, and max_vals to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
        max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
        
        #min_val = np.min(sample, axis=0)
        #max_val = np.max(sample, axis=0)
        
        #normalize
        epsilon = 1e-10
        normalized_sample = 2 * (sample - min_vals.unsqueeze(1)) / (max_vals.unsqueeze(1) - min_vals.unsqueeze(1) + epsilon) - 1
        #normalized_sample = 2 * (sample - min_vals) / (max_vals - min_vals + 1e-10) - 1
        return normalized_sample, label

train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


[VAIQ_NOTE]: Loading NNDCT kernels...


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
"""class ModifiedToyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='nearest')  # set mode = "nearest"
        self.conv = torch.nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu = torch.nn.ReLU()
        self.adaptive_avg_pool2d = torch.nn.AdaptiveAvgPool2d(output_size=1)
        
    def forward(self, x):
        x = self.upsample(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.adaptive_avg_pool2d(x)
        x = x.permute(0, 2, 3, 1) # insert a permute to cancel out the permute inserted by quantizer.
        x = x.reshape(x.size(0), -1)
        return x

# initialize model
num_classes = 2 #BPSK / QPSK
model = CNN1D(num_classes)
"""
import torch
import torch.nn as nn

class CNN2D(nn.Module):
    def __init__(self, num_classes):
        super(CNN2D, self).__init__()
        
        in_channels = 2
        # For the 2D conversion, we're assuming height=1 for the input
        height = 1
        width = 128  # Original length dimension
        
        # Upsample (adjusted for 2D)
        self.upsample = nn.Upsample(scale_factor=(1, 2), mode='bilinear', align_corners=False)  # height stays the same, width is doubled
        
        # Define 2D convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(1, 3), padding=(0, 1))
        self.relu1 = nn.ReLU()
        #self.maxpool1 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool1 = nn.AdaptiveAvgPool2d((1,64))
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 3), padding=(0, 1))
        self.relu2 = nn.ReLU()
        #self.maxpool2 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool2 = nn.AdaptiveAvgPool2d((1,32))
        
        # Adaptive pooling (2D)
        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        
        # Define the fully connected layers (adjusted for adaptive pooling output)
        self.fc1 = nn.Linear(128, 256)  # The input features after pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # Adjust input shape for 2D processing
        x = x.unsqueeze(-2)  # Insert a height dimension of size 1
        
        # Upsample
        x = self.upsample(x)
        
        # 2D convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        #x = self.maxpool1(x)
        x = self.adaptive_pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        #x = self.maxpool2(x)
        x = self.adaptive_pool2(x)
        
        # Adaptive average pooling
        x = self.adaptive_avg_pool2d(x)
        
        # Flatten (before fully connected layers)
        x = x.reshape(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x


model = CNN2D(2)
print(model) #model info

CNN2D(
  (upsample): Upsample(scale_factor=(1.0, 2.0), mode=bilinear)
  (conv1): Conv2d(2, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu1): ReLU()
  (adaptive_pool1): AdaptiveAvgPool2d(output_size=(1, 64))
  (conv2): Conv2d(64, 128, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu2): ReLU()
  (adaptive_pool2): AdaptiveAvgPool2d(output_size=(1, 32))
  (adaptive_avg_pool2d): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


In [3]:
# Specify a target name or fingerprint you want to deploy on
#target = "DPUCAHX8L_ISA0_SP"
target = "DPUCVDX8G_ISA3_C32B6"
#DPUCVDX8G
# Initialize inspector with target
inspector = Inspector(target)
# Note: visualization of inspection results relies on the dot engine.If you don't install dot successfully, set 'image_format = None' when inspecting.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN2D(2)

#dummy_data, labels = train_loader
#data_dummy = []
for data, labels in test_loader:
    data_dummy = data
    break
print(type(data_dummy))
print(data_dummy.shape)
dummy_input = torch.randn(1, 2, 128)
print(dummy_input)

inspector.inspect(model, (data_dummy.clone().detach().requires_grad_(True)), device=device, output_dir="inspect", image_format=None) 


[VAIQ_NOTE]: Inspector is on.
<class 'torch.Tensor'>
torch.Size([1, 2, 128])
tensor([[[ 1.0149e+00,  1.4608e+00,  2.0315e-01,  5.6265e-01, -1.8442e+00,
          -5.6668e-03, -2.0368e+00,  1.9582e+00, -7.8275e-01,  3.5210e-01,
           2.2935e-01,  1.6061e+00, -1.6079e+00, -4.4768e-02,  1.1101e+00,
          -2.1317e-01,  1.1971e+00, -5.1669e-01, -1.2304e+00,  1.6521e+00,
           7.5707e-01, -1.7329e-01,  5.3350e-01, -1.8472e-01, -1.1563e+00,
           8.8880e-01, -2.0746e-01, -3.2721e-01, -8.8517e-01, -1.3579e+00,
           7.1644e-02,  1.2365e-01,  5.0571e-01, -6.0315e-01,  9.8665e-01,
           2.6903e+00,  6.1650e-01, -5.2282e-01, -2.4536e-01,  1.2964e+00,
          -4.2035e-01,  2.0227e+00,  1.8897e+00,  4.3012e-01,  3.4044e-01,
          -5.8531e-01,  3.4126e-01,  1.2555e-03,  1.4521e+00,  4.5152e-01,
           1.2925e+00,  1.2163e-01,  1.6660e-01, -1.2091e+00,  2.8613e-02,
           1.8832e+00,  3.3858e-01, -1.1996e+00,  9.4292e-01,  6.0256e-01,
           2.6056e-01,

/tmp/ipykernel_96712/2094375231.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_96712/2094375231.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_96712/2094375231.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)



[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_CNN2D_ed is torch.nn.Module.

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 16/16 [00:00<00:00, 1713.62it/s, OpInfo: name = return_0, type = Return]                                                    


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(inspect/CNN2D.py)

[VAIQ_WARN]: CNN2D::507 is not tensor.

[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Conv2d[conv1]/ret.7, op type:nndct_conv2d, output shape: [1, 1, 256, 64]
node name:CNN2D::CNN2D/ReLU[relu1]/ret.9, op type:nndct_relu, output shape: [1, 1, 256, 64]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Conv2d[conv2]/ret.11, op type:nndct_conv2d, output shape: [1, 1, 64, 128]
node name:CNN2D::CNN2D/ReLU[relu2]/ret.13, op type:nndct_relu, output shape: [1, 1, 64, 128]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Linear[fc1]/ret.19, op type:nndct_dense, output shape: [1, 256]
node name:CNN2D::CNN2D/ReLU[relu3]/ret.21, op type:nndct_relu, output shape: [1, 256]



[VAIQ_NOTE]: Find subgraph for reshape_fix_1:
node name:CNN2D::CNN2D/ret.3, op type:nndct_reshape, output shape: [1, 2, 1, 128]



[VAIQ_NOTE]


I20240329 19:58:59.373432 96712 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240329 19:58:59.373451 96712 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240329 19:58:59.373456 96712 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCVDX8G_ISA3_C32B6
I20240329 19:58:59.373512 96712 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_conv2d_nndct_relu_8xEnwILaZC3Wtdif, with op num: 9
I20240329 19:58:59.373517 96712 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240329 19:58:59.376130 96712 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20240329 19:58:59.376164 96712 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240329 19:58:59.379510 96712 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240329 19:58:59.379520 96712 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240329 19:58:59.379524 96712 compi

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

def train(model):

    learning_rate = 0.001
    num_epochs = 3

    # loss function and optimizer definition
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # training
    for epoch in range(num_epochs):
        model.train()  # Set the model in training mode

        for data, labels in train_loader:
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(data)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

        # print loss
        print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}')
        
    return model

def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            print(outputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # predicted and actual labels
            for i in range(len(labels)):
                print(f'Predicted: {predicted[i]}, Actual: {labels[i]}')
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy




In [5]:
torch.save(model, "trainedModel")

In [6]:
from pytorch_nndct.apis import torch_quantizer, dump_xmodel


model_trained = train(model)
evaluate(model_trained)

quantizer = torch_quantizer("test", model_trained, (data_dummy)) #needs to be in test mode to generate x_model
quant_model = quantizer.quant_model # is quantize aware
#quant_model = train(quant_model)
evaluate(quant_model)
#accl_gen, acc4_gen, loss_get = evaluate(quant_model, val_loader, criterion)
# evaluate the model on test data


/tmp/ipykernel_96712/2094375231.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_96712/2094375231.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_96712/2094375231.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


Epoch [1/3] Loss: 0.0000
Epoch [2/3] Loss: 0.0000
Epoch [3/3] Loss: 0.0000
tensor([[ 4.9650, -5.2172]])
Predicted: 0, Actual: 0
tensor([[-1.7678,  1.4944]])
Predicted: 1, Actual: 1
tensor([[ 0.4613, -0.6061]])
Predicted: 0, Actual: 0
tensor([[-0.2923,  0.1299]])
Predicted: 1, Actual: 1
tensor([[ 0.3762, -0.6467]])
Predicted: 0, Actual: 0
tensor([[ 6.6781, -7.0745]])
Predicted: 0, Actual: 0
tensor([[-0.3421,  0.1078]])
Predicted: 1, Actual: 1
tensor([[ 3.3543, -3.5738]])
Predicted: 0, Actual: 0
tensor([[ 0.0850, -0.2594]])
Predicted: 0, Actual: 1
tensor([[ 4.2642, -4.2843]])
Predicted: 0, Actual: 0
tensor([[-0.5262,  0.2742]])
Predicted: 1, Actual: 1
tensor([[ 5.1700, -5.2123]])
Predicted: 0, Actual: 0
tensor([[ 4.2481, -4.4475]])
Predicted: 0, Actual: 0
tensor([[-0.2624, -0.0900]])
Predicted: 1, Actual: 1
tensor([[-0.6497,  0.4748]])
Predicted: 1, Actual: 1
tensor([[-1.1812,  0.8476]])
Predicted: 1, Actual: 1
tensor([[-0.7803,  0.6090]])
Predicted: 1, Actual: 1
tensor([[-0.9870,  0.768


[VAIQ_NOTE]: OS and CPU information:
               system --- Linux
                 node --- seakn-ThinkPad-P16s-Gen-1
              release --- 6.5.0-26-generic
              version --- #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 12 10:22:43 UTC 2
              machine --- x86_64
            processor --- x86_64

[VAIQ_NOTE]: Tools version information:
                  GCC --- GCC 7.5.0
               python --- 3.8.6
              pytorch --- 1.13.1
        vai_q_pytorch --- 3.5.0+60df3f1+torch1.13.1

[VAIQ_WARN][QUANTIZER_TORCH_CUDA_UNAVAILABLE]: CUDA (HIP) is not available, change device to CPU.

[VAIQ_NOTE]: Quant config file is empty, use default quant configuration

[VAIQ_NOTE]: Quantization test process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing CNN2D...

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_CNN2D_ed is torch.nn.Module.

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 16/16 [00:00<00:00, 2377.22it/s, OpInfo: name = return_0, type = Return]                                                    


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quantize_result/CNN2D.py)

[VAIQ_NOTE]: =>Get module with quantization.
tensor([[ 5.0625, -5.2500]])
Predicted: 0, Actual: 0
tensor([[-1.6875,  1.4375]])
Predicted: 1, Actual: 1
tensor([[ 0.5625, -0.6875]])
Predicted: 0, Actual: 0
tensor([[-0.1875,  0.0625]])
Predicted: 1, Actual: 1
tensor([[ 0.4375, -0.6250]])
Predicted: 0, Actual: 0
tensor([[ 6.4375, -6.7500]])
Predicted: 0, Actual: 0
tensor([[-0.2500,  0.0625]])
Predicted: 1, Actual: 1
tensor([[ 3.5000, -3.6875]])
Predicted: 0, Actual: 0
tensor([[ 0.2500, -0.3125]])
Predicted: 0, Actual: 1
tensor([[ 4.3750, -4.3125]])
Predicted: 0, Actual: 0
tensor([[-0.4375,  0.2500]])
Predicted: 1, Actual: 1
tensor([[ 5.2500, -5.2500]])
Predicted: 0, Actual: 0
tensor([[ 4.3750, -4.5000]])
Predicted: 0, Actual: 0
tensor([[-0.1250, -0.1250]])
Predicted: 0, Actual: 1
tensor([[-0.5625,  0.4375]])
Predicted: 1, Actual: 1
tensor([[-1.1250,  0.8125]])
Predicted

tensor([[ 6.9375, -7.2500]])
Predicted: 0, Actual: 0
tensor([[ 2.3125, -2.3750]])
Predicted: 0, Actual: 0
tensor([[-0.6875,  0.5625]])
Predicted: 1, Actual: 1
tensor([[ 4.0625, -4.1875]])
Predicted: 0, Actual: 0
tensor([[ 1.3125, -1.3750]])
Predicted: 0, Actual: 0
tensor([[ 3.8750, -4.0000]])
Predicted: 0, Actual: 0
tensor([[ 0.1250, -0.4375]])
Predicted: 0, Actual: 1
tensor([[-0.8125,  0.6875]])
Predicted: 1, Actual: 1
tensor([[-0.7500,  0.3750]])
Predicted: 1, Actual: 1
tensor([[-0.1875,  0.0625]])
Predicted: 1, Actual: 0
tensor([[-1.2500,  1.1250]])
Predicted: 1, Actual: 1
tensor([[-0.4375,  0.2500]])
Predicted: 1, Actual: 1
tensor([[-1.3125,  1.1875]])
Predicted: 1, Actual: 1
tensor([[ 0.1250, -0.3750]])
Predicted: 0, Actual: 1
tensor([[-1.4375,  1.2500]])
Predicted: 1, Actual: 1
tensor([[-0.6875,  0.5625]])
Predicted: 1, Actual: 1
tensor([[-0.8750,  0.7500]])
Predicted: 1, Actual: 1
tensor([[ 4.4375, -4.6250]])
Predicted: 0, Actual: 0
tensor([[ 4.2500, -4.4375]])
Predicted: 0, Act

tensor([[-0.8125,  0.5625]])
Predicted: 1, Actual: 1
tensor([[ 1.0625, -1.1250]])
Predicted: 0, Actual: 1
tensor([[ 5.3750, -5.5625]])
Predicted: 0, Actual: 0
tensor([[ 0.0000, -0.1875]])
Predicted: 0, Actual: 0
tensor([[-0.5000,  0.3125]])
Predicted: 1, Actual: 1
tensor([[ 4.9375, -5.1875]])
Predicted: 0, Actual: 0
tensor([[ 5.7500, -5.6250]])
Predicted: 0, Actual: 0
tensor([[ 5.6250, -5.5625]])
Predicted: 0, Actual: 0
tensor([[-0.3125,  0.1875]])
Predicted: 1, Actual: 1
tensor([[-1.0625,  0.9375]])
Predicted: 1, Actual: 1
tensor([[ 5.1250, -5.3125]])
Predicted: 0, Actual: 0
tensor([[ 5.8125, -6.0625]])
Predicted: 0, Actual: 0
tensor([[ 4.3125, -4.5000]])
Predicted: 0, Actual: 0
tensor([[-0.5625,  0.2500]])
Predicted: 1, Actual: 1
tensor([[ 5.3750, -5.5625]])
Predicted: 0, Actual: 0
tensor([[ 0.5000, -0.6250]])
Predicted: 0, Actual: 1
tensor([[ 2.0000, -2.1250]])
Predicted: 0, Actual: 0
tensor([[ 6.1250, -6.0000]])
Predicted: 0, Actual: 0
tensor([[ 6.3750, -6.6250]])
Predicted: 0, Act

89.25

In [7]:
#quantizer.export_quant_config() # creates quantizer config.json

#quantizer.export_torch_script() #exports [model_name]_int.pt
#quantizer.export_onnx_model() #exports [model_name]_int.onnx
quantizer.export_xmodel(deploy_check=False) # in therory exports xmodel, but exports [model_name].py



[VAIQ_NOTE]: =>Converting to xmodel ...

[VAIQ_WARN]: CNN2D::507 is not tensor.

[VAIQ_NOTE]: =>Successfully convert 'CNN2D' to xmodel.(quantize_result/CNN2D_int.xmodel)


In [8]:
data_for_board = []
labels_for_board = []
for data, labels in test_loader:
    data_for_board.append(data)
    labels_for_board.append(labels)
def tensors_to_np(tensor_list):
    return np.array([tensor.numpy() for tensor in tensor_list])
data_for_board_np = tensors_to_np(data_for_board)
labels_for_board_np = tensors_to_np(labels_for_board)


/tmp/ipykernel_96712/2094375231.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_96712/2094375231.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_96712/2094375231.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


In [9]:
with open('data_for_board', 'wb') as file:
    np.savez(file, data = data_for_board_np, labels = labels_for_board_np)

In [10]:
with np.load('data_for_board', allow_pickle=True) as file:
    data = file['data']
    labels = file['labels']
    
data[0]
labels[0]
    

array([0])